In [1]:
import glob
import h5py
import matplotlib.pyplot as plt
import numpy as np
import os
import scipy

from neuropacks import NHP, PVC11, DREAM
from sklearn.linear_model import LinearRegression

%matplotlib inline

# V1

In [11]:
base_path = os.path.join(os.environ['HOME'],
                         'data/pvc11/data/spikes_gratings')
monkey_paths = glob.glob(os.path.join(base_path, '*.mat'))

In [12]:
pvcs = [PVC11(data_path=monkey_path)
        for monkey_path in monkey_paths]

In [13]:
Xs = [pvc.get_design_matrix(form='cosine2')
      for pvc in pvcs]
Ys = [pvc.get_response_matrix(transform='square_root')
      for pvc in pvcs]

In [14]:
preferences_list = []

In [15]:
for pvc, X, Y in zip(pvcs, Xs, Ys):
    ols = LinearRegression()
    ols.fit(X, Y)
    _, preferences = pvc.get_tuning_modulation_and_preference(form='cosine2',
                                                              tuning_coefs=ols.coef_)
    preferences_list.append(preferences)

In [18]:
np.save('v1_preferences.npy', preferences)

# M1

In [2]:
base_path = '/storage/data/nhp'
#base_path = os.path.join(os.environ['HOME'], 'data/nhp')
monkey_paths = glob.glob(os.path.join(base_path, '*.mat'))

In [3]:
nhps = [NHP(data_path=monkey_path)
        for monkey_path in monkey_paths]

In [6]:
preferences = []

for nhp in nhps:
    Y = nhp.get_response_matrix(bin_width=0.15,  transform='square_root')
    Y = Y[:, np.sum(Y, axis=0) != 0]
    
    x = nhp.cursor_pos[0]
    y = nhp.cursor_pos[1]
    t = nhp.timestamps

    dx = np.ediff1d(x)
    dy = np.ediff1d(y)
    dt = np.ediff1d(t)

    vx = dx / dt
    vy = dy / dt
    
    x_binned = scipy.stats.binned_statistic(
        np.arange(x.size),
        x,
        statistic='mean',
        bins=Y.shape[0]).statistic

    y_binned = scipy.stats.binned_statistic(
        np.arange(y.size),
        y,
        statistic='mean',
        bins=Y.shape[0]).statistic

    angles = np.rad2deg(np.arctan2(y_binned, x_binned))

    vx_binned = scipy.stats.binned_statistic(
        np.arange(vx.size),
        vx,
        statistic='mean',
        bins=Y.shape[0]).statistic

    vy_binned = scipy.stats.binned_statistic(
        np.arange(vy.size),
        vy,
        statistic='mean',
        bins=Y.shape[0]).statistic
    
    X = np.vstack((np.cos(angles), np.sin(angles), vx_binned, vy_binned)).T
    
    ols = LinearRegression()
    ols.fit(X, Y)
    
    _, prefs = DREAM.get_tuning_modulation_and_preference(
        form='cosine',
        tuning_coefs=ols.coef_[:, :2])
    
    preferences.append(prefs)

In [ ]:
np.save('m1_preferences.npy', preferences)